In [1]:
import pandas as pd 
import numpy as np 
import configparser
import os

import nltk 

config = configparser.ConfigParser()
config.read("env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
base_path = config['DEFAULT']['base_path']
code_dir = config['DEFAULT']['code_dir']

data_prefix = 'Maha'

OHCO = ['book_id','chap_id','sec_id', 'para_num', 'sent_num', 'token_num']

os.chdir(code_dir)
import preprocess
import bow_analysis
os.chdir(base_path)

In [2]:
book_dict = {}
lib_data = []
ohco_level = {k:pd.DataFrame() for k in OHCO}

all_dir_list = [f for f in os.listdir(data_home) if f.endswith('.txt')]
for book_id, book_file in enumerate(all_dir_list):
    # Raw File Path
    raw_file_path = f"{data_home}/{all_dir_list[book_id]}"

    # Getting the OHCO Level tables for this book
    book_dict[book_id+1] = preprocess.create_tokendf(raw_file_path)

    # Inserting book_id for each OHCO Level and appending each book in its OHCO ID
    for ohco_id in OHCO[1:]:
        # print(ohco_id)
        book_dict[book_id+1][ohco_id] = book_dict[book_id+1][ohco_id].reset_index()
        book_dict[book_id+1][ohco_id]['book_id'] = book_id + 1
        book_dict[book_id+1][ohco_id] = book_dict[book_id+1][ohco_id].set_index(OHCO[:OHCO.index(ohco_id)+1])
        # book_dict[book_id+1][ohco_id].index = pd.MultiIndex.from_tuples(
        #                                     [(book_id+1,) + idx for idx in book_dict[book_id+1][ohco_id].index],
        #                                     names=['book_id'] + book_dict[book_id+1][ohco_id].index.names)
        ohco_level[ohco_id] = pd.concat([ohco_level[ohco_id], book_dict[book_id+1][ohco_id]])



    print(f"{str(book_id+1).zfill(2)}] {book_file} - {book_dict[book_id+1]['book_id']}")

    lib_data.append((book_id+1, all_dir_list[book_id], book_dict[book_id+1]['book_id']))


for ohco_id in OHCO[1:]:
    ohco_level[ohco_id].to_csv(f"{output_dir}/OHCO/{data_prefix}-{ohco_id}.csv", sep = "|")




01] maha01.txt - Adi Parva
02] maha02.txt - Sabha Parva
03] maha03.txt - Vana Parva
04] maha04.txt - Virata Parva
05] maha05.txt - Udyoga Parva
06] maha06.txt - Bhishma Parva
07] maha07.txt - Drona Parva
08] maha08.txt - Karna Parva
09] maha09.txt - Shalya Parva
10] maha10.txt - Sauptika Parva
11] maha11.txt - Stri Parva
12] maha12.txt - Santi Parva
13] maha13.txt - Anusasana Parva
14] maha14.txt - Aswamedha Parva
15] maha15.txt - Asramavasika Parva
16] maha16.txt - Mausala Parva
17] maha17.txt - Mahaprasthanika Parva
18] maha18.txt - Svargarohanika Parva


In [3]:
book_dict[book_id+1].keys()

dict_keys(['book_id', 'chap_id', 'sec_id', 'para_num', 'sent_num', 'token_num'])

In [4]:
ohco_level['sec_id'].head(10)

section_str
book_id chap_id sec_id                                                   
1       1       1       (Anukramanika Parva)\nOm! Having bowed down to...
        2       1       (Sangraha Parva)\n"The Rishis said, 'O son of ...
        3       2       (Paushya Parva)\nSauti said, "Janamejaya, the ...
        4       3       (Pauloma Parva)\n'UGRASRAVA SAUTI, the son of ...
                4       (Pauloma Parva continued)\n"Saunaka said, 'Chi...
                5       (Pauloma Parva continued)\n"Sauti said, 'O Bra...
                6       (Pauloma Parva continued)\n"Sauti said, 'the g...
                7       (Pauloma Parva continued)\n"Sauti said, 'O Bra...
                8       (Pauloma Parva continued)\n"Sauti said, 'While...
                9       (Pauloma Parva continued)\nSauti said, 'And Ru...

In [5]:
CORPUS = ohco_level['token_num']
CORPUS = CORPUS[CORPUS.term_str != '']
CORPUS = CORPUS[CORPUS.term_str.isnull()==False]
CORPUS = CORPUS[CORPUS.token_str.isnull()==False]
CORPUS['pos_group'] = CORPUS.pos.str[:2]
CORPUS.to_csv(fr"{output_dir}\F2\{data_prefix}-CORPUS.csv", sep = '|')
CORPUS

pos_tuple  pos  \
book_id chap_id sec_id para_num sent_num token_num                             
1       1       1      0        0        1          (Anukramanika, NNP)  NNP   
                                         2                 (Parva, NNP)  NNP   
                                         4                    (Om, NNP)  NNP   
                                1        0                (Having, VBG)  VBG   
                                         1                 (bowed, VBN)  VBN   
...                                                                 ...  ...   
18      1       5      0        147      25                    (of, IN)   IN   
                                         26               (Vishnu, NNP)  NNP   
                                         27                  (like, IN)   IN   
                                         28               (Vishnu, NNP)  NNP   
                                         29              (himself, PRP)  PRP   

                                                       token_str  \
book_id chap_id sec_id para_num sent_num token_num                 
1       1       1      0        0        1          Anukramanika   
                                         2                 Parva   
                                         4                    Om   
                                1        0                Having   
                                         1                 bowed   
...                                                          ...   
18      1       5      0        147      25                   of   
                                         26               Vishnu   
                                         27                 like   
                                         28               Vishnu   
                                         29              himself   

                                                        term_str pos_group  
book_id chap_id sec_id para_num sent_num token_num                          
1       1       1      0        0        1          anukramanika        NN  
                                         2                 parva        NN  
                                         4                    om        NN  
                                1        0                having        VB  
                                         1                 bowed        VB  
...                                                          ...       ...  
18      1       5      0        147      25                   of        IN  
                                         26               vishnu        NN  
                                         27                 like        IN  
                                         28               vishnu        NN  
                                         29              himself        PR  

[2409433 rows x 5 columns]

In [6]:
genre_df = pd.read_json(f"{data_home}/lib.json")
genre_df = genre_df.set_index("book_id")
genre_df

,parva_name,timeline,key_focus,main_theme,broad_theme
book_id,,,,,
1,Adi,Pre-War,"Origins, family lineages, childhood of heroes",lineage,Mythology
2,Sabha,Pre-War,"Dice game, humiliation of Draupadi, exile begins",power,Strategy
3,Vana,Pre-War,"Forest exile, philosophical tales, sub-stories",philosophy,Philosophy
4,Virata,Pre-War,Incognito year in King Virata's court,strategy,Strategy
5,Udyoga,Pre-War,"Failed diplomacy, war preparations",diplomacy,Philosophy
6,Bhishma,War,"Start of war, Krishna's counsel, Bhagavad Gita",dharma,Philosophy
7,Drona,War,"Drona leads Kauravas, death of Abhimanyu",sacrifice,Strategy
8,Karna,War,"Karna becomes commander, dramatic duels",loyalty,Strategy
9,Shalya,War,"Final battles, Shalya leads, Duryodhana defeated",conflict,War


In [7]:
CORPUS['term_len'] = CORPUS['token_str'].apply(len)
CORPUS

pos_tuple  pos  \
book_id chap_id sec_id para_num sent_num token_num                             
1       1       1      0        0        1          (Anukramanika, NNP)  NNP   
                                         2                 (Parva, NNP)  NNP   
                                         4                    (Om, NNP)  NNP   
                                1        0                (Having, VBG)  VBG   
                                         1                 (bowed, VBN)  VBN   
...                                                                 ...  ...   
18      1       5      0        147      25                    (of, IN)   IN   
                                         26               (Vishnu, NNP)  NNP   
                                         27                  (like, IN)   IN   
                                         28               (Vishnu, NNP)  NNP   
                                         29              (himself, PRP)  PRP   

                                                       token_str  \
book_id chap_id sec_id para_num sent_num token_num                 
1       1       1      0        0        1          Anukramanika   
                                         2                 Parva   
                                         4                    Om   
                                1        0                Having   
                                         1                 bowed   
...                                                          ...   
18      1       5      0        147      25                   of   
                                         26               Vishnu   
                                         27                 like   
                                         28               Vishnu   
                                         29              himself   

                                                        term_str pos_group  \
book_id chap_id sec_id para_num sent_num token_num                           
1       1       1      0        0        1          anukramanika        NN   
                                         2                 parva        NN   
                                         4                    om        NN   
                                1        0                having        VB   
                                         1                 bowed        VB   
...                                                          ...       ...   
18      1       5      0        147      25                   of        IN   
                                         26               vishnu        NN   
                                         27                 like        IN   
                                         28               vishnu        NN   
                                         29              himself        PR   

                                                    term_len  
book_id chap_id sec_id para_num sent_num token_num            
1       1       1      0        0        1                12  
                                         2                 5  
                                         4                 2  
                                1        0                 6  
                                         1                 5  
...                                                      ...  
18      1       5      0        147      25                2  
                                         26                6  
                                         27                4  
                                         28                6  
                                         29                7  

[2409433 rows x 6 columns]

In [8]:
LIB = pd.DataFrame(lib_data, columns=['book_id','source_file_path','raw_title'])\
    .set_index('book_id').sort_index()

LIB['book_len'] = CORPUS.groupby('book_id').term_str.count()

LIB['n_chaps'] = CORPUS.reset_index()[['book_id','chap_id']]\
    .drop_duplicates()\
    .groupby('book_id').chap_id.count()

LIB['n_secs'] = CORPUS.reset_index()[['book_id','chap_id','sec_id']]\
    .drop_duplicates()\
    .groupby('book_id').sec_id.count()

LIB['n_sents'] = CORPUS.reset_index()[['book_id','chap_id','sec_id','sent_num']]\
    .drop_duplicates()\
    .groupby('book_id').sent_num.count()

LIB['n_tokens'] = CORPUS.reset_index()[['book_id','chap_id','sec_id','sent_num','token_num']]\
    .drop_duplicates()\
    .groupby('book_id').token_num.count()

LIB['n_chars'] = CORPUS.groupby('book_id')['term_len'].sum()


LIB = LIB.join(genre_df)

LIB.to_csv(f"{output_dir}\F2\{data_prefix}-LIB.csv", sep = "|")
LIB

<string>:27: SyntaxWarning: invalid escape sequence '\{'
<>:27: SyntaxWarning: invalid escape sequence '\{'
<>:27: SyntaxWarning: invalid escape sequence '\F'
<string>:27: SyntaxWarning: invalid escape sequence '\{'
<>:27: SyntaxWarning: invalid escape sequence '\{'
<>:27: SyntaxWarning: invalid escape sequence '\F'
C:\Users\vishg\AppData\Local\Temp\ipykernel_17984\1747485705.py:27: SyntaxWarning: invalid escape sequence '\{'
  LIB.to_csv(f"{output_dir}\F2\{data_prefix}-LIB.csv", sep = "|")
C:\Users\vishg\AppData\Local\Temp\ipykernel_17984\1747485705.py:27: SyntaxWarning: invalid escape sequence '\F'
  LIB.to_csv(f"{output_dir}\F2\{data_prefix}-LIB.csv", sep = "|")
<string>:27: SyntaxWarning: invalid escape sequence '\{'


,source_file_path,raw_title,book_len,n_chaps,n_secs,n_sents,n_tokens,n_chars,parva_name,timeline,key_focus,main_theme,broad_theme
book_id,,,,,,,,,,,,,
1,maha01.txt,Adi Parva,228684,19,235,12422,228684,1055885,Adi,Pre-War,"Origins, family lineages, childhood of heroes",lineage,Mythology
2,maha02.txt,Sabha Parva,74955,10,78,3766,74955,351427,Sabha,Pre-War,"Dice game, humiliation of Draupadi, exile begins",power,Strategy
3,maha03.txt,Vana Parva,323206,21,312,16645,323206,1475333,Vana,Pre-War,"Forest exile, philosophical tales, sub-stories",philosophy,Philosophy
4,maha04.txt,Virata Parva,61159,4,71,2893,61159,282557,Virata,Pre-War,Incognito year in King Virata's court,strategy,Strategy
5,maha05.txt,Udyoga Parva,188731,10,198,9605,188731,867032,Udyoga,Pre-War,"Failed diplomacy, war preparations",diplomacy,Philosophy
6,maha06.txt,Bhishma Parva,148383,4,123,7481,148383,699763,Bhishma,War,"Start of war, Krishna's counsel, Bhagavad Gita",dharma,Philosophy
7,maha07.txt,Drona Parva,245962,8,199,12384,245962,1164914,Drona,War,"Drona leads Kauravas, death of Abhimanyu",sacrifice,Strategy
8,maha08.txt,Karna Parva,138246,2,95,6455,138246,651818,Karna,War,"Karna becomes commander, dramatic duels",loyalty,Strategy
9,maha09.txt,Shalya Parva,91545,4,64,4636,91545,432464,Shalya,War,"Final battles, Shalya leads, Duryodhana defeated",conflict,War


In [9]:
LIB.sum()

source_file_path    maha01.txtmaha02.txtmaha03.txtmaha04.txtmaha05...
raw_title           Adi ParvaSabha ParvaVana ParvaVirata ParvaUdyo...
book_len                                                      2409433
n_chaps                                                            99
n_secs                                                           2089
n_sents                                                        126870
n_tokens                                                      2409433
n_chars                                                      11166915
parva_name          AdiSabhaVanaVirataUdyogaBhishmaDronaKarnaShaly...
timeline            Pre-WarPre-WarPre-WarPre-WarPre-WarWarWarWarWa...
key_focus           Origins, family lineages, childhood of heroesD...
main_theme          lineagepowerphilosophystrategydiplomacydharmas...
broad_theme         MythologyStrategyPhilosophyStrategyPhilosophyP...
dtype: object

In [21]:
VOCAB = CORPUS.term_str.value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)
VOCAB['s'] = 1 / VOCAB['p']
VOCAB['h'] = VOCAB['p'] * VOCAB['i']

sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1


VOCAB['stop'] = VOCAB.index.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['stem_porter'] = VOCAB.apply(lambda x: stemmer1.stem(x.name), 1)

from nltk.stem.snowball import SnowballStemmer
stemmer2 = SnowballStemmer("english")
VOCAB['stem_snowball'] = VOCAB.apply(lambda x: stemmer2.stem(x.name), 1)

from nltk.stem.lancaster import LancasterStemmer
stemmer3 = LancasterStemmer()
VOCAB['stem_lancaster'] = VOCAB.apply(lambda x: stemmer3.stem(x.name), 1)


# Getting the PoS from CORPUS
VOCAB['max_pos'] = CORPUS[['term_str','pos']].value_counts().unstack(fill_value=0).idxmax(1)

# Getting the Most frequent PoS Group
VOCAB['max_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().unstack(fill_value=0).idxmax(1)

# Set of all Possible PoS Groups
VOCAB['n_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().unstack().count(1)
VOCAB['cat_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().to_frame('n').reset_index()\
    .groupby('term_str').pos_group.apply(lambda x: set(x))

# Set of all Possible PoS
VOCAB['n_pos'] = CORPUS[['term_str','pos']].value_counts().unstack().count(1)
VOCAB['cat_pos'] = CORPUS[['term_str','pos']].value_counts().to_frame('n').reset_index()\
    .groupby('term_str').pos.apply(lambda x: set(x))


VOCAB.sort_values("n_pos", ascending=0)

VOCAB.to_csv(fr"{output_dir}\F2\{data_prefix}-VOCAB.csv", sep = "|")
VOCAB

,n,n_chars,p,i,s,h,stop,stem_porter,stem_snowball,stem_lancaster,max_pos,max_pos_group,n_pos_group,cat_pos_group,n_pos,cat_pos
term_str,,,,,,,,,,,,,,,,
01,1,2,4.150354e-07,21.200262,2.409433e+06,0.000009,0,01,01,01,CD,CD,1,{CD},1,{CD}
1,15,1,6.225531e-06,17.293372,1.606289e+05,0.000108,0,1,1,1,CD,CD,1,{CD},1,{CD}
10,9,2,3.735319e-06,18.030337,2.677148e+05,0.000067,0,10,10,10,CD,CD,1,{CD},1,{CD}
100,9,3,3.735319e-06,18.030337,2.677148e+05,0.000067,0,100,100,100,CD,CD,1,{CD},1,{CD}
1000,41,4,1.701645e-05,15.842710,5.876666e+04,0.000270,0,1000,1000,1000,CD,CD,1,{CD},1,{CD}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zeal,2,4,8.300708e-07,20.200262,1.204716e+06,0.000017,0,zeal,zeal,zeal,NN,NN,1,{NN},1,{NN}
zealously,4,9,1.660142e-06,19.200262,6.023582e+05,0.000032,0,zealous,zealous,zeal,RB,RB,1,{RB},1,{RB}
zenana,1,6,4.150354e-07,21.200262,2.409433e+06,0.000009,0,zenana,zenana,zenan,NN,NN,1,{NN},1,{NN}


In [11]:
VOCAB[VOCAB.stop==1]

,n,n_chars,p,i,s,h,stop,stem_porter,stem_snowball,stem_lancaster,max_pos,max_pos_group,n_pos_group,cat_pos_group,n_pos,cat_pos
term_str,,,,,,,,,,,,,,,,
a,28160,1,0.011687,6.418903,85.562251,0.075020,1,a,a,a,DT,DT,3,"{NN, VB, DT}",3,"{VB, NNP, DT}"
about,1547,5,0.000642,10.605005,1557.487395,0.006809,1,about,about,about,IN,IN,4,"{NN, IN, RP, RB}",4,"{NN, IN, RP, RB}"
above,269,5,0.000112,13.128800,8957.000000,0.001466,1,abov,abov,abov,IN,IN,5,"{JJ, RB, VB, IN, NN}",5,"{JJ, RB, VB, IN, NN}"
after,3486,5,0.001447,9.432905,691.174125,0.013648,1,after,after,aft,IN,IN,4,"{NN, VB, JJ, IN}",4,"{JJ, VBD, NNP, IN}"
again,1743,5,0.000723,10.432905,1382.348250,0.007547,1,again,again,again,RB,RB,3,"{NN, VB, RB}",3,"{VB, NNP, RB}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
you,1281,3,0.000532,10.877207,1880.900078,0.005783,1,you,you,you,PRP,PR,1,{PR},1,{PRP}
your,657,4,0.000273,11.840513,3667.325723,0.003229,1,your,your,yo,PRP$,PR,2,"{NN, PR}",2,"{NNP, PRP$}"
yours,51,5,0.000021,15.527837,47243.784314,0.000329,1,your,your,yo,NNS,NN,3,"{NN, VB, RB}",7,"{RB, NNP, VBP, NN, VBZ, VBN, NNS}"


In [12]:
CORPUS[CORPUS.pos_group == "''"]

pos_tuple pos  \
book_id chap_id sec_id para_num sent_num token_num                       
1       11      175    0        7        8              (wild, '')  ''   
3       1       1      0        21       7           (whither, '')  ''   
        3       31     0        64       3          (becometh, '')  ''   
4       1       3      0        16       5                (ye, '')  ''   
6       4       117    0        38       6                (ye, '')  ''   
7       5       87     0        9        27             (thou, '')  ''   
9       4       35     0        64       9                (ye, '')  ''   
11      1       11     0        42       7              (thou, '')  ''   
12      2       151    0        46       7                (ye, '')  ''   
                                144      6                (ye, '')  ''   
13      1       16     0        18       5            (middle, '')  ''   
14      2       21     0        37       51             (past, '')  ''   

                                                   token_str  term_str  \
book_id chap_id sec_id para_num sent_num token_num                       
1       11      175    0        7        8              wild      wild   
3       1       1      0        21       7           whither   whither   
        3       31     0        64       3          becometh  becometh   
4       1       3      0        16       5                ye        ye   
6       4       117    0        38       6                ye        ye   
7       5       87     0        9        27             thou      thou   
9       4       35     0        64       9                ye        ye   
11      1       11     0        42       7              thou      thou   
12      2       151    0        46       7                ye        ye   
                                144      6                ye        ye   
13      1       16     0        18       5            middle    middle   
14      2       21     0        37       51             past      past   

                                                   pos_group  term_len  
book_id chap_id sec_id para_num sent_num token_num                      
1       11      175    0        7        8                ''         4  
3       1       1      0        21       7                ''         7  
        3       31     0        64       3                ''         8  
4       1       3      0        16       5                ''         2  
6       4       117    0        38       6                ''         2  
7       5       87     0        9        27               ''         4  
9       4       35     0        64       9                ''         2  
11      1       11     0        42       7                ''         4  
12      2       151    0        46       7                ''         2  
                                144      6                ''         2  
13      1       16     0        18       5                ''         6  
14      2       21     0        37       51               ''         4

In [13]:
ohco_level['chap_id']

chap_str  \
book_id chap_id                                                      
1       1        (Anukramanika Parva)\nOm! Having bowed down to...   
        2        (Sangraha Parva)\n"The Rishis said, 'O son of ...   
        3        (Paushya Parva)\nSauti said, "Janamejaya, the ...   
        4        (Pauloma Parva)\n'UGRASRAVA SAUTI, the son of ...   
        5        (Astika Parva)\n"Saunaka said, 'For what reaso...   
...                                                            ...   
15      2        (Putradarsana Parva)\n"Janamejaya said, 'Tell ...   
        3        (Naradagamana Parva)\n"Vaisampayana said, 'Aft...   
16      1        (Mausala Parva)\nOm! Having bowed down unto Na...   
17      1        (Mahaprasthanika Parva)\nOm! Having bowed down...   
18      1        (Svargarohanika Parva)\nOm! Having bowed down ...   

                             chap_name  
book_id chap_id                         
1       1           Anukramanika Parva  
        2               Sangraha Parva  
        3                Paushya Parva  
        4                Pauloma Parva  
        5                 Astika Parva  
...                                ...  
15      2           Putradarsana Parva  
        3           Naradagamana Parva  
16      1                Mausala Parva  
17      1        Mahaprasthanika Parva  
18      1         Svargarohanika Parva  

[99 rows x 2 columns]

In [14]:
a = ohco_level['chap_id']['chap_name']
a

book_id  chap_id
1        1             Anukramanika Parva
         2                 Sangraha Parva
         3                  Paushya Parva
         4                  Pauloma Parva
         5                   Astika Parva
                            ...          
15       2             Putradarsana Parva
         3             Naradagamana Parva
16       1                  Mausala Parva
17       1          Mahaprasthanika Parva
18       1           Svargarohanika Parva
Name: chap_name, Length: 99, dtype: object

In [15]:
a[a.isnull()==True]

Series([], Name: chap_name, dtype: object)

In [16]:
LIB2 = pd.read_csv(f"{output_dir}/OHCO/{data_prefix}-chap_id.csv", sep = "|")
LIB2 = LIB2.drop("chap_str", axis=1).merge(LIB.reset_index()[['book_id','raw_title','source_file_path']], on="book_id").rename(columns = {'raw_title':"book_name"}).set_index(['book_id','chap_id'])
LIB2

chap_name              book_name source_file_path
book_id chap_id                                                               
1       1           Anukramanika Parva              Adi Parva       maha01.txt
        2               Sangraha Parva              Adi Parva       maha01.txt
        3                Paushya Parva              Adi Parva       maha01.txt
        4                Pauloma Parva              Adi Parva       maha01.txt
        5                 Astika Parva              Adi Parva       maha01.txt
...                                ...                    ...              ...
15      2           Putradarsana Parva     Asramavasika Parva       maha15.txt
        3           Naradagamana Parva     Asramavasika Parva       maha15.txt
16      1                Mausala Parva          Mausala Parva       maha16.txt
17      1        Mahaprasthanika Parva  Mahaprasthanika Parva       maha17.txt
18      1         Svargarohanika Parva   Svargarohanika Parva       maha18.txt

[99 rows x 3 columns]

In [22]:
LIB2['chap_len'] = CORPUS.groupby(['book_id','chap_id']).term_str.count()

LIB2['n_secs'] = CORPUS.reset_index()[['book_id','chap_id','sec_id']]\
    .drop_duplicates()\
    .groupby(['book_id','chap_id']).sec_id.count()

LIB2['n_sents'] = CORPUS.reset_index()[['book_id','chap_id','sec_id','sent_num']]\
    .drop_duplicates()\
    .groupby(['book_id','chap_id']).sent_num.count()

LIB2['n_tokens'] = CORPUS.reset_index()[['book_id','chap_id','sec_id','sent_num','token_num']]\
    .drop_duplicates()\
    .groupby(['book_id','chap_id']).token_num.count()

LIB2.to_csv(fr"{output_dir}\F2\{data_prefix}-LIB2.csv", sep = "|")
LIB2

chap_name              book_name  \
book_id chap_id                                                 
1       1           Anukramanika Parva              Adi Parva   
        2               Sangraha Parva              Adi Parva   
        3                Paushya Parva              Adi Parva   
        4                Pauloma Parva              Adi Parva   
        5                 Astika Parva              Adi Parva   
...                                ...                    ...   
15      2           Putradarsana Parva     Asramavasika Parva   
        3           Naradagamana Parva     Asramavasika Parva   
16      1                Mausala Parva          Mausala Parva   
17      1        Mahaprasthanika Parva  Mahaprasthanika Parva   
18      1         Svargarohanika Parva   Svargarohanika Parva   

                source_file_path  chap_len  n_secs  n_sents  n_tokens  
book_id chap_id                                                        
1       1             maha01.txt      7511       1      374      7511  
        2             maha01.txt      8655       1      524      8655  
        3             maha01.txt      6137       1      396      6137  
        4             maha01.txt      4661       9      261      4661  
        5             maha01.txt     30619      46     1753     30619  
...                          ...       ...     ...      ...       ...  
15      2             maha15.txt      6789       8      417      6789  
        3             maha15.txt      2551       3      149      2551  
16      1             maha16.txt      8072       7      432      8072  
17      1             maha17.txt      3062       2      206      3062  
18      1             maha18.txt      8628       5      495      8628  

[99 rows x 7 columns]

In [18]:
LIB['n_chars'].mean()

620384.1666666666

In [19]:
CORPUS.shape
"|".join(CORPUS.index.names)
"|".join(CORPUS.columns)

'pos_tuple|pos|token_str|term_str|pos_group|term_len'

In [20]:
# VOCAB.shape
# "|".join(CORPUS.index.names)
"|".join(VOCAB.columns)

'n|n_chars|p|i|s|h|stop|stem_porter|stem_snowball|stem_lancaster|max_pos|max_pos_group|n_pos_group|cat_pos_group|n_pos|cat_pos'